In [1]:

import dash
from dash import dcc, html, Input, Output, State, callback
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Load datasets
print("Loading datasets...")
customers = pd.read_csv('customers_eCommerce.csv')
orders = pd.read_csv('orders_eCommerce.csv')
order_items = pd.read_csv('order_items_eCommerce.csv')
order_payments = pd.read_csv('order_payments_eCommerce.csv')
order_reviews = pd.read_csv('order_reviews_eCommerce.csv')
products = pd.read_csv('products_eCommerce.csv')
sellers = pd.read_csv('sellers_eCommerce.csv')
product_category = pd.read_csv('product_category_name_translation_eCommerce.csv')
geolocation = pd.read_csv('geolocation_eCommerce.csv')
leads_qualified = pd.read_csv('leads_qualified_eCommerce.csv')
leads_closed = pd.read_csv('leads_closed_eCommerce.csv')


# Data preprocessing
# Convert date columns to datetime
date_columns = {
    'orders': ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 
               'order_delivered_customer_date', 'order_estimated_delivery_date'],
    'order_reviews': ['review_creation_date', 'review_answer_timestamp'],
    'order_items': ['shipping_limit_date'],
    'leads_qualified': ['first_contact_date'],
    'leads_closed': ['won_date']
}

for df_name, cols in date_columns.items():
    df = locals()[df_name]
    for col in cols:
        if col in df.columns:
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')
            except Exception as e:
                print(f"Could not convert {col} in {df_name} to datetime: {e}")

# Ensure review_score is numeric
order_reviews['review_score'] = pd.to_numeric(order_reviews['review_score'], errors='coerce')

# Calculate delivery metrics
orders['delivery_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days
orders['estimated_delivery_time'] = (orders['order_estimated_delivery_date'] - orders['order_purchase_timestamp']).dt.days
orders['delivery_delay'] = orders['delivery_time'] - orders['estimated_delivery_time']

# Create delivery status categories
orders['delivery_status'] = pd.cut(
    orders['delivery_delay'],
    bins=[-np.inf, -5, 0, 3, 10, np.inf], # Adjusted bins for potential NaNs or large values
    labels=['Very Early', 'Early', 'On Time', 'Late', 'Very Late'],
    right=False # Ensure 0 falls into 'On Time'
)

# Join tables for analysis
orders_with_items = pd.merge(orders, order_items, on='order_id')
orders_customers = pd.merge(orders, customers, on='customer_id')
orders_items_customers = pd.merge(orders_with_items, customers, on='customer_id')

# Join with products and categories
orders_products = pd.merge(orders_with_items, products, on='product_id')
orders_products_cat = pd.merge(orders_products, product_category, on='product_category_name', how='left')

# Join reviews with orders and products
reviews_orders = pd.merge(order_reviews, orders, on='order_id')
reviews_items = pd.merge(reviews_orders, order_items[['order_id', 'product_id']], on='order_id', how='inner') # Ensure only items with reviews
reviews_products = pd.merge(reviews_items, products, on='product_id')
reviews_categories = pd.merge(reviews_products, product_category, on='product_category_name', how='left')

# Extract year-month for time series
orders_with_items['year_month'] = orders_with_items['order_purchase_timestamp'].dt.strftime('%Y-%m')

# Initialize the Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
server = app.server

# Define colors (from original app.py)
colors = {
    'background': '#f9f9f9',
    'text': '#333333',
    'primary': '#2c3e50',
    'secondary': '#3498db',
    'accent': '#e74c3c',
    'success': '#2ecc71',
    'warning': '#f39c12'
}

# Define inline styles based on styles.css
# Note: Pseudo-classes (:hover) and media queries are not directly translatable to inline styles.
#       Transitions might not work as expected without CSS classes.
#       Styles for internal component elements (like .DateInput_input) might not apply correctly via parent style prop.

body_style = {
    'fontFamily': "'Segoe UI', Tahoma, Geneva, Verdana, sans-serif", # From CSS
    'margin': '0',
    'padding': '0',
    'backgroundColor': '#f9f9f9' # From CSS (matches original colors['background'])
}

tabs_container_style = {
    'marginBottom': '20px' # From CSS .dash-tabs-container
}

tab_style = {
    'padding': '15px 20px',
    'fontSize': '16px',
    'fontWeight': '500',
    'borderRadius': '5px 5px 0 0',
    'border': 'none',
    'backgroundColor': '#e6e6e6',
    'color': '#333',
    # 'transition': 'all 0.3s ease' # Transition may not work well inline
}

selected_tab_style = {
    'padding': '15px 20px', # Repeat base style
    'fontSize': '16px', # Repeat base style
    'fontWeight': '500', # Repeat base style
    'borderRadius': '5px 5px 0 0', # Repeat base style
    'border': 'none', # Repeat base style
    'backgroundColor': '#2c3e50', # From CSS .dash-tab--selected
    'color': 'white' # From CSS .dash-tab--selected
}

kpi_card_style = {
    'backgroundColor': 'white',
    'borderRadius': '8px',
    'boxShadow': '0 4px 8px rgba(0,0,0,0.1)',
    'padding': '20px',
    'textAlign': 'center', # Added for centering content
    'flex': '1', # Added for flex layout
    'margin': '0 10px', # Added spacing between cards
    # 'transition': 'all 0.3s ease' # Transition may not work well inline
}

kpi_title_style = {
    'fontSize': '16px',
    'color': '#7f8c8d',
    'marginBottom': '10px',
    'textAlign': 'center'
}

kpi_value_style = {
    'fontSize': '36px',
    'fontWeight': 'bold',
    'textAlign': 'center',
    'color': '#333' # Added default color
}

chart_container_style = {
    'backgroundColor': 'white',
    'borderRadius': '8px',
    'boxShadow': '0 4px 8px rgba(0,0,0,0.1)',
    'padding': '15px',
    'marginBottom': '20px'
}

filter_container_style = {
    # 'backgroundColor': 'white', # Let's keep the main background for filters
    # 'borderRadius': '8px',
    # 'boxShadow': '0 4px 8px rgba(0,0,0,0.1)',
    'padding': '15px',
    'marginBottom': '20px',
    'display': 'flex',
    'alignItems': 'center',
    'justifyContent': 'center'
}

filter_label_style = {
    'fontWeight': '500',
    'marginRight': '15px'
}

dropdown_style = {
    'borderRadius': '4px',
    'border': '1px solid #ddd',
    'width': '300px' # Added width for better layout
}

# Helper function to create KPI cards with inline styles
def create_kpi_card(title, value, color_value=colors['primary']):
    # Use a copy of the base style to avoid modifying the original dict
    card_style = kpi_card_style.copy()
    value_style = kpi_value_style.copy()
    value_style['color'] = color_value # Apply specific color to value
    
    return html.Div([
        html.H4(title, style=kpi_title_style),
        html.H2(value, style=value_style)
    ], style=card_style)

# Create app layout with inline styles
app.layout = html.Div(style=body_style, children=[
    # Header (already uses inline styles)
    html.Div([
        html.H1('eCommerce Analytics Dashboard', 
                style={'textAlign': 'center', 'color': colors['primary'], 'padding': '20px'}),
        html.P('Unlocking Insights to Improve eCommerce Efficiency and Customer Satisfaction',
               style={'textAlign': 'center', 'color': colors['text'], 'fontSize': '18px', 'marginBottom': '20px'})
    ]),
    
    # Navigation tabs with inline styles
    dcc.Tabs(id='tabs', value='tab-overview', children=[
        dcc.Tab(label='Overview', value='tab-overview', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Customer Insights', value='tab-customers', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Sales Performance', value='tab-sales', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Delivery & Logistics', value='tab-delivery', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Reviews & Satisfaction', value='tab-reviews', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Lead Conversion', value='tab-leads', style=tab_style, selected_style=selected_tab_style),
    ], style=tabs_container_style), # Apply container style
    
    # Date range filter with inline styles
    html.Div([
        html.H4('Filter by Date Range:', style=filter_label_style), # Apply label style
        dcc.DatePickerRange(
            id='date-range',
            min_date_allowed=orders['order_purchase_timestamp'].min().date(),
            max_date_allowed=orders['order_purchase_timestamp'].max().date(),
            start_date=orders['order_purchase_timestamp'].min().date(),
            end_date=orders['order_purchase_timestamp'].max().date(),
            # Cannot directly style internal DateInput_input with parent style prop
        ),
    ], style=filter_container_style), # Apply container style
    
    # State filter with inline styles
    html.Div([
        html.H4('Filter by State:', style=filter_label_style), # Apply label style
        dcc.Dropdown(
            id='state-filter',
            options=[{'label': state, 'value': state} for state in sorted(customers['customer_state'].unique())],
            multi=True,
            placeholder='Select states...',
            style=dropdown_style # Apply dropdown style
        ),
    ], style={**filter_container_style, 'width': '50%', 'margin': '0 auto'}), # Merge container style with layout styles
    
    # Content div - will be populated by the callback
    html.Div(id='tab-content')
])

# Callback to update tab content
@app.callback(
    Output('tab-content', 'children'),
    [Input('tabs', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('state-filter', 'value')]
)
def render_tab_content(tab, start_date, end_date, states):
    # Convert string dates to datetime
    if start_date is not None:
        start_date = pd.to_datetime(start_date)
    if end_date is not None:
        end_date = pd.to_datetime(end_date)
    
    # Filter data based on date range
    filtered_orders = orders.copy() # Work with copies to avoid modifying original dataframes
    if start_date is not None and end_date is not None:
        filtered_orders = filtered_orders[(filtered_orders['order_purchase_timestamp'] >= start_date) & 
                                        (filtered_orders['order_purchase_timestamp'] <= end_date)]
    
    # Join with other tables (use filtered orders)
    filtered_orders_items = pd.merge(filtered_orders, order_items, on='order_id', how='inner')
    filtered_orders_customers = pd.merge(filtered_orders, customers, on='customer_id', how='inner')
    
    # Apply state filter if selected
    if states and len(states) > 0:
        filtered_orders_customers = filtered_orders_customers[filtered_orders_customers['customer_state'].isin(states)]
        # Re-filter order_items based on the filtered customers/orders
        order_ids = filtered_orders_customers['order_id'].unique()
        filtered_orders_items = filtered_orders_items[filtered_orders_items['order_id'].isin(order_ids)]
    
    # Calculate key metrics based on filtered data
    total_sales = filtered_orders_items['price'].sum()
    total_orders = filtered_orders['order_id'].nunique()
    total_customers = filtered_orders['customer_id'].nunique()
    avg_order_value = total_sales / total_orders if total_orders > 0 else 0
    
    # Render content based on selected tab
    if tab == 'tab-overview':
        return render_overview_tab(filtered_orders, filtered_orders_items, filtered_orders_customers, 
                                  total_sales, total_orders, total_customers, avg_order_value)
    elif tab == 'tab-customers':
        return render_customers_tab(filtered_orders_customers, filtered_orders_items)
    elif tab == 'tab-sales':
        return render_sales_tab(filtered_orders_items)
    elif tab == 'tab-delivery':
        return render_delivery_tab(filtered_orders_customers)
    elif tab == 'tab-reviews':
        return render_reviews_tab(filtered_orders)
    elif tab == 'tab-leads':
        return render_leads_tab()
    
    return html.Div([html.H3('Tab content not available')])

# Overview Tab
def render_overview_tab(filtered_orders, filtered_orders_items, filtered_orders_customers, 
                       total_sales, total_orders, total_customers, avg_order_value):
    # Calculate average review score
    reviews_data = pd.merge(filtered_orders, order_reviews, on='order_id', how='inner')
    avg_review = reviews_data['review_score'].mean() if not reviews_data.empty else 0
    
    # Monthly sales trend
    if not filtered_orders_items.empty:
        filtered_orders_items['year_month'] = filtered_orders_items['order_purchase_timestamp'].dt.strftime('%Y-%m')
        monthly_sales = filtered_orders_items.groupby('year_month')['price'].sum().reset_index()
        monthly_sales.columns = ['Year-Month', 'Total Sales']
        sales_trend_fig = px.line(monthly_sales, x='Year-Month', y='Total Sales', 
                                 title='Monthly Sales Trend',
                                 labels={'Total Sales': 'Total Sales ($)', 'Year-Month': 'Month'})
    else:
        sales_trend_fig = go.Figure().update_layout(title='Monthly Sales Trend - No Data')

    # Top product categories
    if not filtered_orders_items.empty:
        orders_products = pd.merge(filtered_orders_items, products, on='product_id', how='inner')
        orders_products_cat = pd.merge(orders_products, product_category, on='product_category_name', how='left')
        category_sales = orders_products_cat.groupby('product_category_name_english')['price'].sum().reset_index()
        category_sales.columns = ['Category', 'Total Sales']
        top_categories = category_sales.sort_values('Total Sales', ascending=False).head(10)
        categories_fig = px.bar(top_categories, y='Category', x='Total Sales', 
                               title='Top 10 Product Categories by Sales',
                               labels={'Total Sales': 'Total Sales ($)', 'Category': 'Product Category'},
                               orientation='h')
    else:
        categories_fig = go.Figure().update_layout(title='Top 10 Product Categories by Sales - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}

    return html.Div([
        # KPI Cards - Use helper function
        html.Div([
            create_kpi_card('Total Sales', f'${total_sales:,.2f}', colors['primary']),
            create_kpi_card('Total Orders', f'{total_orders:,}', colors['secondary']),
            create_kpi_card('Total Customers', f'{total_customers:,}', colors['accent']),
            create_kpi_card('Avg Order Value', f'${avg_order_value:,.2f}', colors['success']),
            create_kpi_card('Avg Review Score', f'{avg_review:.2f}/5' if avg_review else 'N/A', colors['warning']),
        ], style={'display': 'flex', 'justifyContent': 'space-around', 'marginBottom': '30px', 'flexWrap': 'wrap'}), # Added wrap
        
        # Charts with inline styled containers
        html.Div([
            html.Div([dcc.Graph(figure=sales_trend_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=categories_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'flexWrap': 'wrap'}) # Added wrap
    ])

# Customer Insights Tab
def render_customers_tab(filtered_orders_customers, filtered_orders_items):
    # Customer distribution by state
    if not filtered_orders_customers.empty:
        customer_by_state = filtered_orders_customers['customer_state'].value_counts().reset_index()
        customer_by_state.columns = ['State', 'Customer Count']
        state_fig = px.bar(customer_by_state.head(10), x='State', y='Customer Count',
                          title='Top 10 States by Customer Count',
                          color='Customer Count',
                          color_continuous_scale=px.colors.sequential.Blues)
    else:
        state_fig = go.Figure().update_layout(title='Top 10 States by Customer Count - No Data')

    # Customer distribution by city (top 20)
    if not filtered_orders_customers.empty:
        customer_by_city = filtered_orders_customers['customer_city'].value_counts().reset_index().head(20)
        customer_by_city.columns = ['City', 'Customer Count']
        city_fig = px.bar(customer_by_city, x='City', y='Customer Count',
                         title='Top 20 Cities by Customer Count',
                         color='Customer Count',
                         color_continuous_scale=px.colors.sequential.Greens)
    else:
        city_fig = go.Figure().update_layout(title='Top 20 Cities by Customer Count - No Data')

    # Customer segmentation
    if not filtered_orders_customers.empty and not filtered_orders_items.empty:
        customer_frequency = filtered_orders_customers.groupby('customer_id')['order_id'].nunique().reset_index() # Use nunique for orders
        customer_frequency.columns = ['customer_id', 'Purchase Frequency']
        
        customer_value = filtered_orders_items.groupby('customer_id')['price'].sum().reset_index()
        customer_value.columns = ['customer_id', 'Total Spend']
        
        customer_segments = pd.merge(customer_frequency, customer_value, on='customer_id')
        customer_segments['Average Order Value'] = customer_segments['Total Spend'] / customer_segments['Purchase Frequency']
        
        # Sample data for scatter plot if large
        sample_size = min(1000, len(customer_segments))
        segment_sample = customer_segments.sample(sample_size) if sample_size > 0 else customer_segments

        segment_fig = px.scatter(segment_sample,
                                x='Purchase Frequency', y='Average Order Value',
                                size='Total Spend', color='Total Spend',
                                title='Customer Segmentation (Sample)',
                                labels={'Purchase Frequency': 'Purchase Frequency (# Orders)',
                                       'Average Order Value': 'Avg Order Value ($)',
                                       'Total Spend': 'Total Customer Spend ($)'},
                                color_continuous_scale=px.colors.sequential.Viridis,
                                hover_name=segment_sample['customer_id'])
    else:
        segment_fig = go.Figure().update_layout(title='Customer Segmentation - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row

    return html.Div([
        html.Div([
            html.Div([dcc.Graph(figure=state_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=city_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '20px', 'flexWrap': 'wrap'}),
        
        html.Div([dcc.Graph(figure=segment_fig)], style=full_width_chart_style)
    ])

# Sales Performance Tab
def render_sales_tab(filtered_orders_items):
    # Join with products and categories
    if not filtered_orders_items.empty:
        orders_products = pd.merge(filtered_orders_items, products, on='product_id', how='inner')
        orders_products_cat = pd.merge(orders_products, product_category, on='product_category_name', how='left')
        orders_products_cat['product_category_name_english'] = orders_products_cat['product_category_name_english'].fillna('Unknown')

        # Monthly sales trend (already calculated in overview, reuse logic if needed or recalculate)
        filtered_orders_items['year_month'] = filtered_orders_items['order_purchase_timestamp'].dt.strftime('%Y-%m')
        monthly_sales = filtered_orders_items.groupby('year_month')['price'].sum().reset_index()
        monthly_sales.columns = ['Year-Month', 'Total Sales']
        sales_trend_fig = px.line(monthly_sales, x='Year-Month', y='Total Sales', 
                                 title='Monthly Sales Trend',
                                 labels={'Total Sales': 'Total Sales ($)', 'Year-Month': 'Month'})

        # Sales by Category
        category_sales = orders_products_cat.groupby('product_category_name_english')['price'].sum().reset_index()
        category_sales.columns = ['Category', 'Total Sales']
        category_sales = category_sales.sort_values('Total Sales', ascending=False)
        category_sales_fig = px.bar(category_sales.head(15), x='Category', y='Total Sales',
                                   title='Top 15 Product Categories by Sales',
                                   color='Total Sales',
                                   color_continuous_scale=px.colors.sequential.Plasma)

        # Sales by Product (Top 20)
        product_sales = orders_products_cat.groupby('product_id')['price'].agg(['sum', 'count']).reset_index()
        product_sales.columns = ['product_id', 'Total Sales', 'Units Sold']
        product_sales = pd.merge(product_sales, products[['product_id', 'product_category_name']], on='product_id', how='left')
        # Add product name if available - Assuming product name is not directly in products.csv
        # If product names were available, they would be added here
        product_sales = product_sales.sort_values('Total Sales', ascending=False)
        product_sales_fig = px.bar(product_sales.head(20), x='product_id', y='Total Sales',
                                  hover_data=['Units Sold'],
                                  title='Top 20 Products by Sales',
                                  labels={'product_id': 'Product ID'})

    else:
        sales_trend_fig = go.Figure().update_layout(title='Monthly Sales Trend - No Data')
        category_sales_fig = go.Figure().update_layout(title='Top 15 Product Categories by Sales - No Data')
        product_sales_fig = go.Figure().update_layout(title='Top 20 Products by Sales - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row

    return html.Div([
        html.Div([dcc.Graph(figure=sales_trend_fig)], style=full_width_chart_style),
        html.Div([
            html.Div([dcc.Graph(figure=category_sales_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=product_sales_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '20px', 'flexWrap': 'wrap'})
    ])

# Delivery & Logistics Tab
def render_delivery_tab(filtered_orders_customers):
    # Average delivery delay by state
    if not filtered_orders_customers.empty:
        delivery_by_state = filtered_orders_customers.groupby('customer_state')['delivery_delay'].mean().reset_index()
        delivery_by_state.columns = ['State', 'Average Delivery Delay (days)']
        delivery_by_state = delivery_by_state.sort_values('Average Delivery Delay (days)', ascending=False)
        delay_state_fig = px.bar(delivery_by_state, x='State', y='Average Delivery Delay (days)',
                                title='Average Delivery Delay by State',
                                color='Average Delivery Delay (days)',
                                color_continuous_scale=px.colors.sequential.RdBu_r)

        # Delivery status distribution
        delivery_status_counts = filtered_orders_customers['delivery_status'].value_counts(normalize=True).reset_index()
        delivery_status_counts.columns = ['Delivery Status', 'Percentage']
        delivery_status_counts['Percentage'] *= 100 # Convert to percentage
        status_dist_fig = px.pie(delivery_status_counts, names='Delivery Status', values='Percentage',
                                title='Distribution of Delivery Status',
                                hole=0.3)

        # Delivery Time vs. Estimated Time
        # Sample for scatter plot if too large
        sample_size = min(5000, len(filtered_orders_customers))
        delivery_sample = filtered_orders_customers.sample(sample_size) if sample_size > 0 else filtered_orders_customers
        delivery_time_fig = px.scatter(delivery_sample,
                                     x='estimated_delivery_time', y='delivery_time',
                                     color='delivery_delay',
                                     title='Actual vs. Estimated Delivery Time (Sample)',
                                     labels={'estimated_delivery_time': 'Estimated Delivery Time (days)',
                                             'delivery_time': 'Actual Delivery Time (days)',
                                             'delivery_delay': 'Delivery Delay (days)'},
                                     color_continuous_scale=px.colors.sequential.Bluered_r)
        delivery_time_fig.add_shape(type="line", x0=0, y0=0, x1=delivery_sample['estimated_delivery_time'].max(), y1=delivery_sample['estimated_delivery_time'].max(), line=dict(color="Black", width=2, dash="dash"))

    else:
        delay_state_fig = go.Figure().update_layout(title='Average Delivery Delay by State - No Data')
        status_dist_fig = go.Figure().update_layout(title='Distribution of Delivery Status - No Data')
        delivery_time_fig = go.Figure().update_layout(title='Actual vs. Estimated Delivery Time - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row

    return html.Div([
        html.Div([dcc.Graph(figure=delay_state_fig)], style=full_width_chart_style),
        html.Div([
            html.Div([dcc.Graph(figure=status_dist_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=delivery_time_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '20px', 'flexWrap': 'wrap'})
    ])

# Reviews & Satisfaction Tab
def render_reviews_tab(filtered_orders):
    # Join reviews with filtered orders
    reviews_data = pd.merge(filtered_orders, order_reviews, on='order_id', how='inner')
    
    if not reviews_data.empty:
        # Review score distribution
        review_score_dist = reviews_data['review_score'].value_counts().reset_index()
        review_score_dist.columns = ['Review Score', 'Count']
        review_score_dist = review_score_dist.sort_values('Review Score')
        score_dist_fig = px.bar(review_score_dist, x='Review Score', y='Count',
                               title='Distribution of Review Scores',
                               color='Count', color_continuous_scale=px.colors.sequential.YlGnBu)

        # Average review score over time
        reviews_data['review_month'] = reviews_data['review_creation_date'].dt.strftime('%Y-%m')
        monthly_avg_review = reviews_data.groupby('review_month')['review_score'].mean().reset_index()
        monthly_avg_review.columns = ['Month', 'Average Score']
        avg_review_time_fig = px.line(monthly_avg_review, x='Month', y='Average Score',
                                    title='Average Review Score Over Time',
                                    markers=True)

        # Review score by category (using pre-joined reviews_categories)
        # Filter reviews_categories based on filtered_orders
        filtered_review_categories = reviews_categories[reviews_categories['order_id'].isin(filtered_orders['order_id'])]
        if not filtered_review_categories.empty:
            review_by_category = filtered_review_categories.groupby('product_category_name_english')['review_score'].mean().reset_index()
            review_by_category.columns = ['Category', 'Average Review Score']
            review_by_category = review_by_category.sort_values('Average Review Score')
            review_cat_fig = px.bar(review_by_category.head(15), x='Average Review Score', y='Category',
                                   title='Average Review Score by Product Category (Top 15 Lowest)',
                                   orientation='h',
                                   color='Average Review Score',
                                   color_continuous_scale=px.colors.sequential.YlOrRd_r)
        else:
            review_cat_fig = go.Figure().update_layout(title='Average Review Score by Product Category - No Data')

    else:
        score_dist_fig = go.Figure().update_layout(title='Distribution of Review Scores - No Data')
        avg_review_time_fig = go.Figure().update_layout(title='Average Review Score Over Time - No Data')
        review_cat_fig = go.Figure().update_layout(title='Average Review Score by Product Category - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row

    return html.Div([
        html.Div([dcc.Graph(figure=score_dist_fig)], style=full_width_chart_style),
        html.Div([
            html.Div([dcc.Graph(figure=avg_review_time_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=review_cat_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '20px', 'flexWrap': 'wrap'})
    ])

# Lead Conversion Tab
def render_leads_tab():
    # This tab uses separate lead datasets, not affected by main filters
    
    # Lead funnel
    total_qualified = len(leads_qualified)
    total_closed = len(leads_closed)
    # Assuming 'won_date' indicates a won lead
    total_won = len(leads_closed.dropna(subset=['won_date']))
    conversion_rate = (total_won / total_qualified) * 100 if total_qualified > 0 else 0

    funnel_data = pd.DataFrame({
        'Stage': ['Qualified Leads', 'Closed Deals', 'Won Deals'],
        'Count': [total_qualified, total_closed, total_won]
    })
    funnel_fig = px.funnel(funnel_data, x='Count', y='Stage', title='Sales Lead Funnel')

    # Leads over time (Qualified)
    if not leads_qualified.empty:
        leads_qualified['contact_month'] = leads_qualified['first_contact_date'].dt.strftime('%Y-%m')
        monthly_leads = leads_qualified.groupby('contact_month').size().reset_index(name='Count')
        leads_time_fig = px.line(monthly_leads, x='contact_month', y='Count',
                                title='Qualified Leads Over Time',
                                labels={'contact_month': 'Month', 'Count': 'Number of Qualified Leads'},
                                markers=True)
    else:
        leads_time_fig = go.Figure().update_layout(title='Qualified Leads Over Time - No Data')

    # Won deals over time
    if not leads_closed.empty and 'won_date' in leads_closed.columns:
        won_leads = leads_closed.dropna(subset=['won_date']).copy()
        won_leads['won_month'] = won_leads['won_date'].dt.strftime('%Y-%m')
        monthly_won = won_leads.groupby('won_month').size().reset_index(name='Count')
        won_time_fig = px.line(monthly_won, x='won_month', y='Count',
                              title='Won Deals Over Time',
                              labels={'won_month': 'Month', 'Count': 'Number of Won Deals'},
                              markers=True)
    else:
        won_time_fig = go.Figure().update_layout(title='Won Deals Over Time - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row
    kpi_container_style = {'display': 'flex', 'justifyContent': 'center', 'marginBottom': '30px'}

    return html.Div([
        # Lead KPIs
        html.Div([
             create_kpi_card('Total Qualified Leads', f'{total_qualified:,}', colors['primary']),
             create_kpi_card('Total Won Deals', f'{total_won:,}', colors['success']),
             create_kpi_card('Conversion Rate', f'{conversion_rate:.2f}%', colors['secondary']),
        ], style=kpi_container_style),

        html.Div([dcc.Graph(figure=funnel_fig)], style=full_width_chart_style),
        
        html.Div([
            html.Div([dcc.Graph(figure=leads_time_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=won_time_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '20px', 'flexWrap': 'wrap'})
    ])


import dash
from dash import dcc, html, Input, Output, State, callback
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
import os

# Load datasets
print("Loading datasets...")
customers = pd.read_csv('customers_eCommerce.csv')
orders = pd.read_csv('orders_eCommerce.csv')
order_items = pd.read_csv('order_items_eCommerce.csv')
order_payments = pd.read_csv('order_payments_eCommerce.csv')
order_reviews = pd.read_csv('order_reviews_eCommerce.csv')
products = pd.read_csv('products_eCommerce.csv')
sellers = pd.read_csv('sellers_eCommerce.csv')
product_category = pd.read_csv('product_category_name_translation_eCommerce.csv')
geolocation = pd.read_csv('geolocation_eCommerce.csv')
leads_qualified = pd.read_csv('leads_qualified_eCommerce.csv')
leads_closed = pd.read_csv('leads_closed_eCommerce.csv')


# Data preprocessing
# Convert date columns to datetime
date_columns = {
    'orders': ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 
               'order_delivered_customer_date', 'order_estimated_delivery_date'],
    'order_reviews': ['review_creation_date', 'review_answer_timestamp'],
    'order_items': ['shipping_limit_date'],
    'leads_qualified': ['first_contact_date'],
    'leads_closed': ['won_date']
}

for df_name, cols in date_columns.items():
    df = locals()[df_name]
    for col in cols:
        if col in df.columns:
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')
            except Exception as e:
                print(f"Could not convert {col} in {df_name} to datetime: {e}")

# Ensure review_score is numeric
order_reviews['review_score'] = pd.to_numeric(order_reviews['review_score'], errors='coerce')

# Calculate delivery metrics
orders['delivery_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days
orders['estimated_delivery_time'] = (orders['order_estimated_delivery_date'] - orders['order_purchase_timestamp']).dt.days
orders['delivery_delay'] = orders['delivery_time'] - orders['estimated_delivery_time']

# Create delivery status categories
orders['delivery_status'] = pd.cut(
    orders['delivery_delay'],
    bins=[-np.inf, -5, 0, 3, 10, np.inf], # Adjusted bins for potential NaNs or large values
    labels=['Very Early', 'Early', 'On Time', 'Late', 'Very Late'],
    right=False # Ensure 0 falls into 'On Time'
)

# Join tables for analysis
orders_with_items = pd.merge(orders, order_items, on='order_id')
orders_customers = pd.merge(orders, customers, on='customer_id')
orders_items_customers = pd.merge(orders_with_items, customers, on='customer_id')

# Join with products and categories
orders_products = pd.merge(orders_with_items, products, on='product_id')
orders_products_cat = pd.merge(orders_products, product_category, on='product_category_name', how='left')

# Join reviews with orders and products
reviews_orders = pd.merge(order_reviews, orders, on='order_id')
reviews_items = pd.merge(reviews_orders, order_items[['order_id', 'product_id']], on='order_id', how='inner') # Ensure only items with reviews
reviews_products = pd.merge(reviews_items, products, on='product_id')
reviews_categories = pd.merge(reviews_products, product_category, on='product_category_name', how='left')

# Extract year-month for time series
orders_with_items['year_month'] = orders_with_items['order_purchase_timestamp'].dt.strftime('%Y-%m')

# Initialize the Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
server = app.server

# Define colors (from original app.py)
colors = {
    'background': '#f9f9f9',
    'text': '#333333',
    'primary': '#2c3e50',
    'secondary': '#3498db',
    'accent': '#e74c3c',
    'success': '#2ecc71',
    'warning': '#f39c12'
}

# Define inline styles based on styles.css
# Note: Pseudo-classes (:hover) and media queries are not directly translatable to inline styles.
#       Transitions might not work as expected without CSS classes.
#       Styles for internal component elements (like .DateInput_input) might not apply correctly via parent style prop.

body_style = {
    'fontFamily': "'Segoe UI', Tahoma, Geneva, Verdana, sans-serif", 
    'margin': '0',
    'padding': '0',
    'backgroundColor': '#f9f9f9' 
}

tabs_container_style = {
    'marginBottom': '20px' 
}

tab_style = {
    'padding': '15px 20px',
    'fontSize': '16px',
    'fontWeight': '500',
    'borderRadius': '5px 5px 0 0',
    'border': 'none',
    'backgroundColor': '#e6e6e6',
    'color': '#333',
    # 'transition': 'all 0.3s ease' # Transition may not work well inline
}

selected_tab_style = {
    'padding': '15px 20px', # Repeat base style
    'fontSize': '16px', # Repeat base style
    'fontWeight': '500', # Repeat base style
    'borderRadius': '5px 5px 0 0', # Repeat base style
    'border': 'none', # Repeat base style
    'backgroundColor': '#2c3e50', #
    'color': 'white' 
}

kpi_card_style = {
    'backgroundColor': 'white',
    'borderRadius': '8px',
    'boxShadow': '0 4px 8px rgba(0,0,0,0.1)',
    'padding': '20px',
    'textAlign': 'center', # Added for centering content
    'flex': '1', # Added for flex layout
    'margin': '0 10px', # Added spacing between cards
    # 'transition': 'all 0.3s ease' # Transition may not work well inline
}

kpi_title_style = {
    'fontSize': '16px',
    'color': '#7f8c8d',
    'marginBottom': '10px',
    'textAlign': 'center'
}

kpi_value_style = {
    'fontSize': '36px',
    'fontWeight': 'bold',
    'textAlign': 'center',
    'color': '#333' # Added default color
}

chart_container_style = {
    'backgroundColor': 'white',
    'borderRadius': '8px',
    'boxShadow': '0 4px 8px rgba(0,0,0,0.1)',
    'padding': '15px',
    'marginBottom': '20px'
}

filter_container_style = {
    # 'backgroundColor': 'white', # Let's keep the main background for filters
    # 'borderRadius': '8px',
    # 'boxShadow': '0 4px 8px rgba(0,0,0,0.1)',
    'padding': '15px',
    'marginBottom': '20px',
    'display': 'flex',
    'alignItems': 'center',
    'justifyContent': 'center'
}

filter_label_style = {
    'fontWeight': '500',
    'marginRight': '15px'
}

dropdown_style = {
    'borderRadius': '4px',
    'border': '1px solid #ddd',
    'width': '300px' # Added width for better layout
}

# Helper function to create KPI cards with inline styles
def create_kpi_card(title, value, color_value=colors['primary']):
    # Use a copy of the base style to avoid modifying the original dict
    card_style = kpi_card_style.copy()
    value_style = kpi_value_style.copy()
    value_style['color'] = color_value # Apply specific color to value
    
    return html.Div([
        html.H4(title, style=kpi_title_style),
        html.H2(value, style=value_style)
    ], style=card_style)

# Create app layout with inline styles
app.layout = html.Div(style=body_style, children=[
    # Header (already uses inline styles)
    html.Div([
        html.H1('eCommerce Analytics Dashboard', 
                style={'textAlign': 'center', 'color': colors['primary'], 'padding': '20px'}),
        html.P('Unlocking Insights to Improve eCommerce Efficiency and Customer Satisfaction',
               style={'textAlign': 'center', 'color': colors['text'], 'fontSize': '18px', 'marginBottom': '20px'})
    ]),
    
    # Navigation tabs with inline styles
    dcc.Tabs(id='tabs', value='tab-overview', children=[
        dcc.Tab(label='Overview', value='tab-overview', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Customer Insights', value='tab-customers', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Sales Performance', value='tab-sales', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Delivery & Logistics', value='tab-delivery', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Reviews & Satisfaction', value='tab-reviews', style=tab_style, selected_style=selected_tab_style),
        dcc.Tab(label='Lead Conversion', value='tab-leads', style=tab_style, selected_style=selected_tab_style),
    ], style=tabs_container_style), # Apply container style
    
    # Date range filter with inline styles
    html.Div([
        html.H4('Filter by Date Range:', style=filter_label_style), # Apply label style
        dcc.DatePickerRange(
            id='date-range',
            min_date_allowed=orders['order_purchase_timestamp'].min().date(),
            max_date_allowed=orders['order_purchase_timestamp'].max().date(),
            start_date=orders['order_purchase_timestamp'].min().date(),
            end_date=orders['order_purchase_timestamp'].max().date(),
            # Cannot directly style internal DateInput_input with parent style prop
        ),
    ], style=filter_container_style), # Apply container style
    
    # State filter with inline styles
    html.Div([
        html.H4('Filter by State:', style=filter_label_style), # Apply label style
        dcc.Dropdown(
            id='state-filter',
            options=[{'label': state, 'value': state} for state in sorted(customers['customer_state'].unique())],
            multi=True,
            placeholder='Select states...',
            style=dropdown_style # Apply dropdown style
        ),
    ], style={**filter_container_style, 'width': '50%', 'margin': '0 auto'}), # Merge container style with layout styles
    
    # Content div - will be populated by the callback
    html.Div(id='tab-content')
])

# Callback to update tab content
@app.callback(
    Output('tab-content', 'children'),
    [Input('tabs', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('state-filter', 'value')]
)
def render_tab_content(tab, start_date, end_date, states):
    # Convert string dates to datetime
    if start_date is not None:
        start_date = pd.to_datetime(start_date)
    if end_date is not None:
        end_date = pd.to_datetime(end_date)
    
    # Filter data based on date range
    filtered_orders = orders.copy() # Work with copies to avoid modifying original dataframes
    if start_date is not None and end_date is not None:
        filtered_orders = filtered_orders[(filtered_orders['order_purchase_timestamp'] >= start_date) & 
                                        (filtered_orders['order_purchase_timestamp'] <= end_date)]
    
    # Join with other tables (use filtered orders)
    filtered_orders_items = pd.merge(filtered_orders, order_items, on='order_id', how='inner')
    filtered_orders_customers = pd.merge(filtered_orders, customers, on='customer_id', how='inner')
    
    # Apply state filter if selected
    if states and len(states) > 0:
        filtered_orders_customers = filtered_orders_customers[filtered_orders_customers['customer_state'].isin(states)]
        # Re-filter order_items based on the filtered customers/orders
        order_ids = filtered_orders_customers['order_id'].unique()
        filtered_orders_items = filtered_orders_items[filtered_orders_items['order_id'].isin(order_ids)]
    
    # Calculate key metrics based on filtered data
    total_sales = filtered_orders_items['price'].sum()
    total_orders = filtered_orders['order_id'].nunique()
    total_customers = filtered_orders['customer_id'].nunique()
    avg_order_value = total_sales / total_orders if total_orders > 0 else 0
    
    # Render content based on selected tab
    if tab == 'tab-overview':
        return render_overview_tab(filtered_orders, filtered_orders_items, filtered_orders_customers, 
                                  total_sales, total_orders, total_customers, avg_order_value)
    elif tab == 'tab-customers':
        return render_customers_tab(filtered_orders_customers, filtered_orders_items)
    elif tab == 'tab-sales':
        return render_sales_tab(filtered_orders_items)
    elif tab == 'tab-delivery':
        return render_delivery_tab(filtered_orders_customers)
    elif tab == 'tab-reviews':
        return render_reviews_tab(filtered_orders)
    elif tab == 'tab-leads':
        return render_leads_tab()
    
    return html.Div([html.H3('Tab content not available')])

# Overview Tab
def render_overview_tab(filtered_orders, filtered_orders_items, filtered_orders_customers, 
                       total_sales, total_orders, total_customers, avg_order_value):
    # Calculate average review score
    reviews_data = pd.merge(filtered_orders, order_reviews, on='order_id', how='inner')
    avg_review = reviews_data['review_score'].mean() if not reviews_data.empty else 0
    
    # Monthly sales trend
    if not filtered_orders_items.empty:
        filtered_orders_items['year_month'] = filtered_orders_items['order_purchase_timestamp'].dt.strftime('%Y-%m')
        monthly_sales = filtered_orders_items.groupby('year_month')['price'].sum().reset_index()
        monthly_sales.columns = ['Year-Month', 'Total Sales']
        sales_trend_fig = px.line(monthly_sales, x='Year-Month', y='Total Sales', 
                                 title='Monthly Sales Trend',
                                 labels={'Total Sales': 'Total Sales ($)', 'Year-Month': 'Month'})
    else:
        sales_trend_fig = go.Figure().update_layout(title='Monthly Sales Trend - No Data')

    # Top product categories
    if not filtered_orders_items.empty:
        orders_products = pd.merge(filtered_orders_items, products, on='product_id', how='inner')
        orders_products_cat = pd.merge(orders_products, product_category, on='product_category_name', how='left')
        category_sales = orders_products_cat.groupby('product_category_name_english')['price'].sum().reset_index()
        category_sales.columns = ['Category', 'Total Sales']
        top_categories = category_sales.sort_values('Total Sales', ascending=False).head(10)
        categories_fig = px.bar(top_categories, y='Category', x='Total Sales', 
                               title='Top 10 Product Categories by Sales',
                               labels={'Total Sales': 'Total Sales ($)', 'Category': 'Product Category'},
                               orientation='h')
    else:
        categories_fig = go.Figure().update_layout(title='Top 10 Product Categories by Sales - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}

    return html.Div([
        # KPI Cards - Use helper function
        html.Div([
            create_kpi_card('Total Sales', f'${total_sales:,.2f}', colors['primary']),
            create_kpi_card('Total Orders', f'{total_orders:,}', colors['secondary']),
            create_kpi_card('Total Customers', f'{total_customers:,}', colors['accent']),
            create_kpi_card('Avg Order Value', f'${avg_order_value:,.2f}', colors['success']),
            create_kpi_card('Avg Review Score', f'{avg_review:.2f}/5' if avg_review else 'N/A', colors['warning']),
        ], style={'display': 'flex', 'justifyContent': 'space-around', 'marginBottom': '30px', 'flexWrap': 'wrap'}), # Added wrap
        
        # Charts with inline styled containers
        html.Div([
            html.Div([dcc.Graph(figure=sales_trend_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=categories_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'flexWrap': 'wrap'}) # Added wrap
    ])

# Customer Insights Tab
def render_customers_tab(filtered_orders_customers, filtered_orders_items):
    # Customer distribution by state
    if not filtered_orders_customers.empty:
        customer_by_state = filtered_orders_customers['customer_state'].value_counts().reset_index()
        customer_by_state.columns = ['State', 'Customer Count']
        state_fig = px.bar(customer_by_state.head(10), x='State', y='Customer Count',
                          title='Top 10 States by Customer Count',
                          color='Customer Count',
                          color_continuous_scale=px.colors.sequential.Blues)
    else:
        state_fig = go.Figure().update_layout(title='Top 10 States by Customer Count - No Data')

    # Customer distribution by city (top 20)
    if not filtered_orders_customers.empty:
        customer_by_city = filtered_orders_customers['customer_city'].value_counts().reset_index().head(20)
        customer_by_city.columns = ['City', 'Customer Count']
        city_fig = px.bar(customer_by_city, x='City', y='Customer Count',
                         title='Top 20 Cities by Customer Count',
                         color='Customer Count',
                         color_continuous_scale=px.colors.sequential.Greens)
    else:
        city_fig = go.Figure().update_layout(title='Top 20 Cities by Customer Count - No Data')

    # Customer segmentation
    if not filtered_orders_customers.empty and not filtered_orders_items.empty:
        customer_frequency = filtered_orders_customers.groupby('customer_id')['order_id'].nunique().reset_index() # Use nunique for orders
        customer_frequency.columns = ['customer_id', 'Purchase Frequency']
        
        customer_value = filtered_orders_items.groupby('customer_id')['price'].sum().reset_index()
        customer_value.columns = ['customer_id', 'Total Spend']
        
        customer_segments = pd.merge(customer_frequency, customer_value, on='customer_id')
        customer_segments['Average Order Value'] = customer_segments['Total Spend'] / customer_segments['Purchase Frequency']
        
        # Sample data for scatter plot if large
        sample_size = min(1000, len(customer_segments))
        segment_sample = customer_segments.sample(sample_size) if sample_size > 0 else customer_segments

        segment_fig = px.scatter(segment_sample,
                                x='Purchase Frequency', y='Average Order Value',
                                size='Total Spend', color='Total Spend',
                                title='Customer Segmentation (Sample)',
                                labels={'Purchase Frequency': 'Purchase Frequency (# Orders)',
                                       'Average Order Value': 'Avg Order Value ($)',
                                       'Total Spend': 'Total Customer Spend ($)'},
                                color_continuous_scale=px.colors.sequential.Viridis,
                                hover_name=segment_sample['customer_id'])
    else:
        segment_fig = go.Figure().update_layout(title='Customer Segmentation - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row

    return html.Div([
        html.Div([
            html.Div([dcc.Graph(figure=state_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=city_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '20px', 'flexWrap': 'wrap'}),
        
        html.Div([dcc.Graph(figure=segment_fig)], style=full_width_chart_style)
    ])

# Sales Performance Tab
def render_sales_tab(filtered_orders_items):
    # Join with products and categories
    if not filtered_orders_items.empty:
        orders_products = pd.merge(filtered_orders_items, products, on='product_id', how='inner')
        orders_products_cat = pd.merge(orders_products, product_category, on='product_category_name', how='left')
        orders_products_cat['product_category_name_english'] = orders_products_cat['product_category_name_english'].fillna('Unknown')

        # Monthly sales trend (already calculated in overview, reuse logic if needed or recalculate)
        filtered_orders_items['year_month'] = filtered_orders_items['order_purchase_timestamp'].dt.strftime('%Y-%m')
        monthly_sales = filtered_orders_items.groupby('year_month')['price'].sum().reset_index()
        monthly_sales.columns = ['Year-Month', 'Total Sales']
        sales_trend_fig = px.line(monthly_sales, x='Year-Month', y='Total Sales', 
                                 title='Monthly Sales Trend',
                                 labels={'Total Sales': 'Total Sales ($)', 'Year-Month': 'Month'})

        # Sales by Category
        category_sales = orders_products_cat.groupby('product_category_name_english')['price'].sum().reset_index()
        category_sales.columns = ['Category', 'Total Sales']
        category_sales = category_sales.sort_values('Total Sales', ascending=False)
        category_sales_fig = px.bar(category_sales.head(15), x='Category', y='Total Sales',
                                   title='Top 15 Product Categories by Sales',
                                   color='Total Sales',
                                   color_continuous_scale=px.colors.sequential.Plasma)

        # Sales by Product (Top 20)
        product_sales = orders_products_cat.groupby('product_id')['price'].agg(['sum', 'count']).reset_index()
        product_sales.columns = ['product_id', 'Total Sales', 'Units Sold']
        product_sales = pd.merge(product_sales, products[['product_id', 'product_category_name']], on='product_id', how='left')
        # Add product name if available - Assuming product name is not directly in products.csv
        # If product names were available, they would be added here
        product_sales = product_sales.sort_values('Total Sales', ascending=False)
        product_sales_fig = px.bar(product_sales.head(20), x='product_id', y='Total Sales',
                                  hover_data=['Units Sold'],
                                  title='Top 20 Products by Sales',
                                  labels={'product_id': 'Product ID'})

    else:
        sales_trend_fig = go.Figure().update_layout(title='Monthly Sales Trend - No Data')
        category_sales_fig = go.Figure().update_layout(title='Top 15 Product Categories by Sales - No Data')
        product_sales_fig = go.Figure().update_layout(title='Top 20 Products by Sales - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row

    return html.Div([
        html.Div([dcc.Graph(figure=sales_trend_fig)], style=full_width_chart_style),
        html.Div([
            html.Div([dcc.Graph(figure=category_sales_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=product_sales_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '20px', 'flexWrap': 'wrap'})
    ])

# Delivery & Logistics Tab
def render_delivery_tab(filtered_orders_customers):
    # Average delivery delay by state
    if not filtered_orders_customers.empty:
        delivery_by_state = filtered_orders_customers.groupby('customer_state')['delivery_delay'].mean().reset_index()
        delivery_by_state.columns = ['State', 'Average Delivery Delay (days)']
        delivery_by_state = delivery_by_state.sort_values('Average Delivery Delay (days)', ascending=False)
        delay_state_fig = px.bar(delivery_by_state, x='State', y='Average Delivery Delay (days)',
                                title='Average Delivery Delay by State',
                                color='Average Delivery Delay (days)',
                                color_continuous_scale=px.colors.sequential.RdBu_r)

        # Delivery status distribution
        delivery_status_counts = filtered_orders_customers['delivery_status'].value_counts(normalize=True).reset_index()
        delivery_status_counts.columns = ['Delivery Status', 'Percentage']
        delivery_status_counts['Percentage'] *= 100 # Convert to percentage
        status_dist_fig = px.pie(delivery_status_counts, names='Delivery Status', values='Percentage',
                                title='Distribution of Delivery Status',
                                hole=0.3)

        # Delivery Time vs. Estimated Time
        # Sample for scatter plot if too large
        sample_size = min(5000, len(filtered_orders_customers))
        delivery_sample = filtered_orders_customers.sample(sample_size) if sample_size > 0 else filtered_orders_customers
        delivery_time_fig = px.scatter(delivery_sample,
                                     x='estimated_delivery_time', y='delivery_time',
                                     color='delivery_delay',
                                     title='Actual vs. Estimated Delivery Time (Sample)',
                                     labels={'estimated_delivery_time': 'Estimated Delivery Time (days)',
                                             'delivery_time': 'Actual Delivery Time (days)',
                                             'delivery_delay': 'Delivery Delay (days)'},
                                     color_continuous_scale=px.colors.sequential.Bluered_r)
        delivery_time_fig.add_shape(type="line", x0=0, y0=0, x1=delivery_sample['estimated_delivery_time'].max(), y1=delivery_sample['estimated_delivery_time'].max(), line=dict(color="Black", width=2, dash="dash"))

    else:
        delay_state_fig = go.Figure().update_layout(title='Average Delivery Delay by State - No Data')
        status_dist_fig = go.Figure().update_layout(title='Distribution of Delivery Status - No Data')
        delivery_time_fig = go.Figure().update_layout(title='Actual vs. Estimated Delivery Time - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row

    return html.Div([
        html.Div([dcc.Graph(figure=delay_state_fig)], style=full_width_chart_style),
        html.Div([
            html.Div([dcc.Graph(figure=status_dist_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=delivery_time_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '20px', 'flexWrap': 'wrap'})
    ])

# Reviews & Satisfaction Tab
def render_reviews_tab(filtered_orders):
    # Join reviews with filtered orders
    reviews_data = pd.merge(filtered_orders, order_reviews, on='order_id', how='inner')
    
    if not reviews_data.empty:
        # Review score distribution
        review_score_dist = reviews_data['review_score'].value_counts().reset_index()
        review_score_dist.columns = ['Review Score', 'Count']
        review_score_dist = review_score_dist.sort_values('Review Score')
        score_dist_fig = px.bar(review_score_dist, x='Review Score', y='Count',
                               title='Distribution of Review Scores',
                               color='Count', color_continuous_scale=px.colors.sequential.YlGnBu)

        # Average review score over time
        reviews_data['review_month'] = reviews_data['review_creation_date'].dt.strftime('%Y-%m')
        monthly_avg_review = reviews_data.groupby('review_month')['review_score'].mean().reset_index()
        monthly_avg_review.columns = ['Month', 'Average Score']
        avg_review_time_fig = px.line(monthly_avg_review, x='Month', y='Average Score',
                                    title='Average Review Score Over Time',
                                    markers=True)

        # Review score by category (using pre-joined reviews_categories)
        # Filter reviews_categories based on filtered_orders
        filtered_review_categories = reviews_categories[reviews_categories['order_id'].isin(filtered_orders['order_id'])]
        if not filtered_review_categories.empty:
            review_by_category = filtered_review_categories.groupby('product_category_name_english')['review_score'].mean().reset_index()
            review_by_category.columns = ['Category', 'Average Review Score']
            review_by_category = review_by_category.sort_values('Average Review Score')
            review_cat_fig = px.bar(review_by_category.head(15), x='Average Review Score', y='Category',
                                   title='Average Review Score by Product Category (Top 15 Lowest)',
                                   orientation='h',
                                   color='Average Review Score',
                                   color_continuous_scale=px.colors.sequential.YlOrRd_r)
        else:
            review_cat_fig = go.Figure().update_layout(title='Average Review Score by Product Category - No Data')

    else:
        score_dist_fig = go.Figure().update_layout(title='Distribution of Review Scores - No Data')
        avg_review_time_fig = go.Figure().update_layout(title='Average Review Score Over Time - No Data')
        review_cat_fig = go.Figure().update_layout(title='Average Review Score by Product Category - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row

    return html.Div([
        html.Div([dcc.Graph(figure=score_dist_fig)], style=full_width_chart_style),
        html.Div([
            html.Div([dcc.Graph(figure=avg_review_time_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=review_cat_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '20px', 'flexWrap': 'wrap'})
    ])

# Lead Conversion Tab
def render_leads_tab():
    # This tab uses separate lead datasets, not affected by main filters
    
    # Lead funnel
    total_qualified = len(leads_qualified)
    total_closed = len(leads_closed)
    # Assuming 'won_date' indicates a won lead
    total_won = len(leads_closed.dropna(subset=['won_date']))
    conversion_rate = (total_won / total_qualified) * 100 if total_qualified > 0 else 0

    funnel_data = pd.DataFrame({
        'Stage': ['Qualified Leads', 'Closed Deals', 'Won Deals'],
        'Count': [total_qualified, total_closed, total_won]
    })
    funnel_fig = px.funnel(funnel_data, x='Count', y='Stage', title='Sales Lead Funnel')

    # Leads over time (Qualified)
    if not leads_qualified.empty:
        leads_qualified['contact_month'] = leads_qualified['first_contact_date'].dt.strftime('%Y-%m')
        monthly_leads = leads_qualified.groupby('contact_month').size().reset_index(name='Count')
        leads_time_fig = px.line(monthly_leads, x='contact_month', y='Count',
                                title='Qualified Leads Over Time',
                                labels={'contact_month': 'Month', 'Count': 'Number of Qualified Leads'},
                                markers=True)
    else:
        leads_time_fig = go.Figure().update_layout(title='Qualified Leads Over Time - No Data')

    # Won deals over time
    if not leads_closed.empty and 'won_date' in leads_closed.columns:
        won_leads = leads_closed.dropna(subset=['won_date']).copy()
        won_leads['won_month'] = won_leads['won_date'].dt.strftime('%Y-%m')
        monthly_won = won_leads.groupby('won_month').size().reset_index(name='Count')
        won_time_fig = px.line(monthly_won, x='won_month', y='Count',
                              title='Won Deals Over Time',
                              labels={'won_month': 'Month', 'Count': 'Number of Won Deals'},
                              markers=True)
    else:
        won_time_fig = go.Figure().update_layout(title='Won Deals Over Time - No Data')

    # Merge chart container style with layout style
    chart_div_style = {**chart_container_style, 'width': '48%'}
    full_width_chart_style = {**chart_container_style, 'width': '98%'} # For single chart row
    kpi_container_style = {'display': 'flex', 'justifyContent': 'center', 'marginBottom': '30px'}

    return html.Div([
        # Lead KPIs
        html.Div([
             create_kpi_card('Total Qualified Leads', f'{total_qualified:,}', colors['primary']),
             create_kpi_card('Total Won Deals', f'{total_won:,}', colors['success']),
             create_kpi_card('Conversion Rate', f'{conversion_rate:.2f}%', colors['secondary']),
        ], style=kpi_container_style),

        html.Div([dcc.Graph(figure=funnel_fig)], style=full_width_chart_style),
        
        html.Div([
            html.Div([dcc.Graph(figure=leads_time_fig)], style=chart_div_style),
            html.Div([dcc.Graph(figure=won_time_fig)], style=chart_div_style),
        ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '20px', 'flexWrap': 'wrap'})
    ])


if __name__ == '__main__':
    print("Starting Dash server...")
    app.run(port=7080)
     
   


Loading datasets...
Loading datasets...
Starting Dash server...
